# Imports

In [4]:
# for working on google colab:
#!wget -q "https://github.com/sh0w/machine-learning-pattern-classfication/raw/main/task3_classification/dataset/dataset_with_predicted_mean_A_mean_V.csv" -O "dataset.csv"
#df = pd.read_csv("dataset.csv")

In [35]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.feature_selection import SelectKBest

In [36]:

# does not contain predicted mean_A and mean_V values:
# df = pd.read_csv("dataset/phase_3_TRAIN_7d499bff69ca69b6_6372c3e_MLPC2021_generic.csv")

# contains our predicted meanA and meanV:
df = pd.read_csv("dataset/dataset_with_score_mode_predicted.csv")


In [37]:
df = df.sample(frac=1, random_state=1337)
df.head()

,id,essentia_dissonance_mean,essentia_dissonance_stdev,essentia_dynamic_complexity,essentia_loudness,essentia_onset_rate,essentia_pitch_salience_mean,essentia_pitch_salience_stdev,essentia_spectral_centroid_mean,essentia_spectral_centroid_stdev,...,midlevel_features_minorness,score_mode,score_key_strength,mean_A,mean_V,quadrant,predicted_mean_A,predicted_mean_V,predicted_quadrant,predicted_score_mode
635,FG-28-002,0.122095,0.064984,4.623885,16.808792,1.9,0.442560,0.124470,1244.223145,450.345795,...,-0.111223,1.0,0.731050,22.758621,-2.379310,3,25.401666,-3.317308,3,1
1305,SR-03-003,0.214142,0.055768,1.130288,419.003784,2.7,0.656047,0.088780,723.109863,95.818260,...,0.163383,1.0,0.697512,89.655172,-2.275862,2,64.835649,-1.420100,2,1
2247,RT-15-001,0.241415,0.092346,7.721171,17.821379,0.6,0.511267,0.120436,6452.331055,1776.684570,...,0.023106,1.0,0.708694,31.034483,-2.034483,3,22.705086,-2.024927,3,1
13,GG-02-000,0.137928,0.084980,5.940224,47.652794,2.8,0.378687,0.153892,2218.609131,1753.583862,...,-0.168097,0.0,0.847891,28.620690,1.413793,4,34.310774,-0.594297,3,0
1863,AS-16-002,0.117757,0.052556,2.940668,60.153103,2.8,0.510892,0.110120,787.357056,171.977753,...,-0.346411,1.0,0.556205,25.862069,-2.379310,3,30.251207,-3.370932,3,1


# Split Dataset to X and y=quadrant

In [38]:
# drop target value, student annotations and string ID from input features:
# all that is left are the music features:
X = df.drop(columns=['quadrant','mean_A','mean_V','id','score_mode','score_key_strength'])

# we want to predict the QUADRANT label:
y = df["quadrant"]

# Perform GridSearchCV to find best parameters for a Neural Network (MLPClassifier)

In [45]:

# Define a pipeline to search for the best combination of PCA truncation
# and classifier regularization.
from sklearn.decomposition import PCA, KernelPCA

mlp_gs = MLPClassifier(max_iter=1000)

parameter_space = [{
    'selector__k': [5,10,20,"all"],
    'nn__hidden_layer_sizes': [(5,10), (10,20,10), (5,7,5), (5,7), (10,5)],
    'nn__activation': ['tanh', 'relu', 'sigmoid'],
    'nn__solver': ['sgd', 'adam', 'lbfgs'],
    'nn__alpha': [0.0005, 0.005, 0.05],
    'nn__learning_rate': ['adaptive','constant']
}]

pipeline = Pipeline([
        ("std_scaler", StandardScaler()),
        ("selector", SelectKBest()),
        ('nn', mlp_gs)
      ])



In [46]:
clf_v = RandomizedSearchCV(pipeline, parameter_space, n_jobs=-1, cv=5, return_train_score=True, verbose=4, n_iter=140)

clf_v.fit(X, y)

Fitting 5 folds for each of 140 candidates, totalling 700 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:   53.8s
[Parallel(n_jobs=-1)]: Done 205 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 376 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-1)]: Done 597 tasks      | elapsed:  8.6min
[Parallel(n_jobs=-1)]: Done 700 out of 700 | elapsed: 10.6min finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('std_scaler',
                                              StandardScaler(copy=True,
                                                             with_mean=True,
                                                             with_std=True)),
                                             ('selector',
                                              SelectKBest(k=10,
                                                          score_func=<function f_classif at 0x102bc0c20>)),
                                             ('nn',
                                              MLPClassifier(activation='relu',
                                                            alpha=0.0001,
                                                            batch_size='auto',
                                                            beta_1=0.9,
                                  

In [47]:
print("\n"+"#"*50,"\nbest estimator: ", clf_v.best_estimator_,"\n"+"#"*50,"\nbest params: ", clf_v.best_params_, "\n"+"#"*50,"\nbest score: ",clf_v.best_score_, "\n")



################################################## 
best estimator:  Pipeline(memory=None,
         steps=[('std_scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('selector',
                 SelectKBest(k=20,
                             score_func=<function f_classif at 0x102bc0c20>)),
                ('nn',
                 MLPClassifier(activation='relu', alpha=0.005,
                               batch_size='auto', beta_1=0.9, beta_2=0.999,
                               early_stopping=False, epsilon=1e-08,
                               hidden_layer_sizes=(10, 5),
                               learning_rate='adaptive',
                               learning_rate_init=0.001, max_fun=15000,
                               max_iter=1000, momentum=0.9, n_iter_no_change=10,
                               nesterovs_momentum=True, power_t=0.5,
                               random_state=None, shuffle=True, solver='adam',
           

In [48]:
quadrants_predicted = clf_v.predict(X)


In [50]:
correct_quadrants_new = sum(quadrants_predicted == df['quadrant']) / len(df)
print("Percentage of Samples with Quadrant correctly predicted:", correct_quadrants_new)

correct_quadrants_old = sum(df['predicted_quadrant'] == df['quadrant']) / len(df)
print("Score of Previous preduction (using only mean_V and mean_V)", correct_quadrants_old)

Percentage of Samples with Quadrant correctly predicted: 0.8273878020713463
Score of Previous preduction (using only mean_V and mean_V) 0.7809742999616417
